In [1]:
import requests
import pandas as pd
import uuid
from datetime import datetime

In [2]:
# Define the REST Countries API endpoint
api_url = "https://restcountries.com/v3.1/all"

# Fetch data from the API
response = requests.get(api_url)
countries_data = response.json()

# Initialize lists to store data for each table
dim_country_data = []
fact_country_statistics_data = []

# Current timestamp for the 'updated_ts' field
updated_ts = datetime.now()

# Loop through each country in the data
for country in countries_data:
    # Generate a UUID for the country
    country_uuid = str(uuid.uuid4())
    
    # Extract data for DimCountry
    dim_country_entry = {
        'country_uuid': country_uuid,
        'name': country.get('name', {}).get('common'),
        'alpha2code': country.get('cca2'),
        'alpha3code': country.get('cca3'),
        'capital': country.get('capital', [None])[0] if country.get('capital') else None,
        'region': country.get('region'),
        'subregion': country.get('subregion'),
        'demonym': country.get('demonyms', {}).get('eng', {}).get('m'),
        'nativename': list(country.get('name', {}).get('nativeName', {}).values())[0].get('common') if country.get('name', {}).get('nativeName') else None,
        'numericcode': country.get('ccn3'),
        'timezones': country.get('timezones'),
        'languages': list(country.get('languages', {}).values()) if country.get('languages') else None,
        'currencies': list(country.get('currencies', {}).keys()) if country.get('currencies') else None,
        'borders': country.get('borders'),
        'topleveldomain': country.get('tld', [None])[0] if country.get('tld') else None,
        'callingcodes': country.get('idd', {}).get('root'),
        'altspellings': country.get('altSpellings'),
        'latlng': country.get('latlng'),
        'translations': {k: v['common'] for k, v in country.get('translations', {}).items()} if country.get('translations') else None,
        'flag': country.get('flags', {}).get('png'),
        'regionalblocs': [bloc.get('acronym') for bloc in country.get('regionalBlocs', [])] if country.get('regionalBlocs') else None,
        'cioc': country.get('cioc'),
        'independent': country.get('independent'),
        'updated_ts': updated_ts
    }
    dim_country_data.append(dim_country_entry)
    
    # Extract data for FactCountryStatistics
    fact_country_entry = {
        'country_uuid': country_uuid,
        'population': country.get('population'),
        'area': country.get('area'),
        'gini': list(country.get('gini', {}).values())[0] if country.get('gini') else None,
        'updated_ts': updated_ts
    }
    fact_country_statistics_data.append(fact_country_entry)

# Create DataFrames
dim_country_df = pd.DataFrame(dim_country_data)
fact_country_statistics_df = pd.DataFrame(fact_country_statistics_data)

# Display the DataFrames
print("DimCountry DataFrame:")
print(dim_country_df.head())

print("FactCountryStatistics DataFrame:")
print(fact_country_statistics_df.head())

DimCountry DataFrame:
                           country_uuid           name alpha2code alpha3code  \
0  dba13cfd-9c2e-48d1-b433-aa10803274de  South Georgia         GS        SGS   
1  f22d6d86-71ca-4cf8-a094-994a33349348        Grenada         GD        GRD   
2  960217e6-6eee-4846-901d-22d5b556de17    Switzerland         CH        CHE   
3  ac7af50d-c325-408e-8e8e-7aa7e40a26e2   Sierra Leone         SL        SLE   
4  4285cf9b-2cdb-45c5-8420-a7c261581eb0        Hungary         HU        HUN   

             capital     region       subregion  \
0  King Edward Point  Antarctic            None   
1       St. George's   Americas       Caribbean   
2               Bern     Europe  Western Europe   
3           Freetown     Africa  Western Africa   
4           Budapest     Europe  Central Europe   

                                  demonym     nativename numericcode  ...  \
0  South Georgian South Sandwich Islander  South Georgia         239  ...   
1                               Gren